### Utils



In [ ]:
#| default_exp utils

In [ ]:
#|export
import torch
from torch import nn

from torch.utils.data import default_collate
from typing import Mapping

In [ ]:
#|export
def_device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

def to_device(x, device=def_device):
    if isinstance(x, torch.Tensor): return x.to(device)
    if isinstance(x, Mapping): return {k:v.to(device) for k,v in x.items()}
    return type(x)(to_device(o, device) for o in x)

def collate_device(b): return to_device(default_collate(b))

In [ ]:
def_device

'mps'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()